In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [10]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
type(data)

pandas.core.frame.DataFrame

In [12]:
# DATA CLEANING

data.drop(['RowNumber', 'Surname', 'CustomerId'], axis=1, inplace=True)

In [13]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
# label transform (encoding)

gender_encoder = LabelEncoder()
data.Gender = gender_encoder.fit_transform(data.Gender)
data.head(), type(data)

(   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
 0          619    France       0   42       2       0.00              1   
 1          608     Spain       0   41       1   83807.86              1   
 2          502    France       0   42       8  159660.80              3   
 3          699    France       0   39       1       0.00              2   
 4          850     Spain       0   43       2  125510.82              1   
 
    HasCrCard  IsActiveMember  EstimatedSalary  Exited  
 0          1               1        101348.88       1  
 1          0               1        112542.58       0  
 2          1               0        113931.57       1  
 3          0               0         93826.63       0  
 4          1               1         79084.10       0  ,
 pandas.core.frame.DataFrame)

In [15]:
from sklearn.preprocessing import OneHotEncoder
geo_encoder = OneHotEncoder()
transformed = geo_encoder.fit_transform(data[['Geography']])

In [16]:
geo_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [17]:
geo_enc_df = pd.DataFrame(transformed.toarray(), columns=geo_encoder.get_feature_names_out(['Geography']))

In [18]:
geo_enc_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [19]:
# FROM MULTIPLE COLS WITH TEXT

# from sklearn.compose import ColumnTransformer
# txt_features = ['Geography'] 
# # encoder = onehot (defined above)
# transformer = ColumnTransformer([('geo_trans', onehot, txt_features)], remainder='drop')


In [20]:
data = pd.concat([data.drop('Geography', axis=1), geo_enc_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
#  save the enocoders

# with open('gender_encoder.pkl', 'wb') as file:
#     pickle.dump(gender_encoder, file)
    
# with open('geo_encoder.pkl', 'wb') as file:
#     pickle.dump(geo_encoder, file)

In [22]:
# DIVIDE DATA INTO TARGET AND FEATURES

X = data.drop('Exited', axis=1)
y = data.Exited

In [23]:
# SPLIT DATA INTO TRAINING AND TEST SETS

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
# SCALING FEATURES

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# save Scaler

# with open('scaler.pkl', 'wb') as file:
#     pickle.dump(scaler, file)

In [26]:
# ANN IMPLEMENTATION

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime;

In [29]:
# Build ann model

model = Sequential([
    
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'), # First hidden layer HL1
    Dense(32, activation='relu'), # Second hidden layer HL2
    Dense(1, activation='sigmoid') # Output layer
    
])

In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile model

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# to change the learning rate of the optimizer
# optm = tf.keras.optimizers.Adam(learning_rate=0.01)
# and use optm in place of 'adam'

In [38]:
# setting up tensorboard

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S') #logs directory
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [39]:
# Setup EarlyStopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [40]:
# Training the model

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    callbacks=[tensorflow_callbacks, early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7804 - loss: 0.4694 - val_accuracy: 0.8350 - val_loss: 0.3891
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8420 - loss: 0.3749 - val_accuracy: 0.8515 - val_loss: 0.3619
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8544 - loss: 0.3519 - val_accuracy: 0.8575 - val_loss: 0.3525
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3443 - val_accuracy: 0.8630 - val_loss: 0.3483
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8601 - loss: 0.3406 - val_accuracy: 0.8615 - val_loss: 0.3477
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8612 - loss: 0.3388 - val_accuracy: 0.8645 - val_loss: 0.3422
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8621 - loss: 0.3351 - val_accuracy: 0.8675 - val_loss: 0.3416
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8627 - loss: 0.3333 - val_accu

In [54]:
# save the model

model.save("model.keras", save_format="keras_v3")


In [43]:
# load tensorboard extention

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 99320), started 0:23:08 ago. (Use '!kill 99320' to kill it.)

In [52]:
print(gender_encoder.classes_)

['Female' 'Male']
